In [2]:
import pandas as pd
df = pd.read_csv("nav_time_series.csv", delimiter=";")
isins_df = df["Scheme Code"].unique().tolist()

C:\Users\vaibh\AppData\Local\Temp\ipykernel_11328\266203744.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("nav_time_series.csv", delimiter=";")


In [27]:
with open("tt.txt", "r") as f:
    isins_file = list(map(int,f.readline().split(",")))

In [28]:
len(isins_df) , len(isins_file)

(21724, 21724)

In [29]:
len(set(isins_file).difference(set(isins_df)))

0

In [21]:
a = set(sorted(isins_df))

In [22]:
b = set(sorted(isins_file))

In [24]:
a_b = a.difference(b)
len(a_b)

16955

In [26]:
b_a = b.difference(a)
len(b_a)

0

In [4]:
import requests

# Target URL
url = "https://www.amfiindia.com/modules/NavHistoryAll"

# Headers to mimic browser
headers = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.amfiindia.com",
    "Referer": "https://www.amfiindia.com/net-asset-value/nav-history",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
}

# Payload with your desired date
data = {
    "fDate": "14-Jul-2025"  # <-- You can change this to any date you want
}

# Send the POST request
response = requests.post(url, headers=headers, data=data)

# Display the response content (HTML)
if response.status_code == 200:
    print("✅ Request successful!")
    print(response.text[:1000])  # Only print first 1000 chars for readability
else:
    print(f"❌ Request failed with status code {response.status_code}")


✅ Request successful!

                <table border="0" cellpadding="0" cellspacing="0" id="divDownloadsAll">
                    <tr>
                        <td>
                            <form action="/modules/_xlsDownload" method="post">	<input id="hdnVal" name="hdnVal" type="hidden" value="0" />	<input type="submit" class="serch-rgt" title="Excel"></form>
                            <form action="/modules/_wordDownload" method="post">	<input id="hdnWordVal" name="hdnWordVal" type="hidden" value="0" />	<input type="submit" class="word-serch-rgt" title="Word"></form>
                        </td>
                    </tr>
                </table>
                <div id="divExcelAll">
                    <table style="margin-top: -25px" class="nav-resut-tble" border="0" cellpadding="0" cellspacing="0">
                        <tbody>
                            <tr style="background-color: rgb(240, 240, 240);">
                                <th colspan="4">View all NAVs for all

In [5]:
with open("tt.html","w") as f :
    f.write(response.text)

In [ ]:
respon